In [1]:
from agentex import Agentex

client = Agentex(base_url="http://localhost:5003")

In [2]:
AGENT_NAME = "at010-agent-chat"

In [3]:
# (REQUIRED) Create a new task. For Agentic agents, you must create a task for messages to be associated with.
import uuid

rpc_response = client.agents.create_task(
    agent_name=AGENT_NAME,
    params={
        "name": f"{str(uuid.uuid4())[:8]}-task",
        "params": {}
    }
)

task = rpc_response.result
print(task)

Task(id='197f2eea-1dae-4107-8f7e-890989d8fa9c', created_at=datetime.datetime(2025, 10, 1, 23, 59, 53, 130494, tzinfo=TzInfo(UTC)), name='a6efd0ee-task', params={}, status='RUNNING', status_reason='Task created, forwarding to ACP server', task_metadata=None, updated_at=datetime.datetime(2025, 10, 1, 23, 59, 53, 130494, tzinfo=TzInfo(UTC)))


In [4]:
# Send an event to the agent

# The response is expected to be a list of TaskMessage objects, which is a union of the following types:
# - TextContent: A message with just text content   
# - DataContent: A message with JSON-serializable data content
# - ToolRequestContent: A message with a tool request, which contains a JSON-serializable request to call a tool
# - ToolResponseContent: A message with a tool response, which contains response object from a tool call in its content

# When processing the message/send response, if you are expecting more than TextContent, such as DataContent, ToolRequestContent, or ToolResponseContent, you can process them as well

rpc_response = client.agents.send_event(
    agent_name=AGENT_NAME,
    params={
        "content": {"type": "text", "author": "user", "content": "Tell me about recent AI news for today only."},
        "task_id": task.id,
    }
)

event = rpc_response.result
print(event)

Event(id='edbed4c7-2e74-4db3-a92d-225af5f566fe', agent_id='2f4d3b3d-6a59-46ff-993e-afd6c1f1c7ab', sequence_id=130, task_id='197f2eea-1dae-4107-8f7e-890989d8fa9c', content=TextContent(author='user', content='Tell me about recent AI news for today only.', attachments=None, format='plain', style='static', type='text'), created_at=datetime.datetime(2025, 10, 1, 23, 59, 53, 175465, tzinfo=TzInfo(UTC)))


In [ ]:
# Subscribe to the async task messages produced by the agent
from agentex.lib.utils.dev_tools import subscribe_to_async_task_messages

task_messages = subscribe_to_async_task_messages(
    client=client,
    task=task, 
    only_after_timestamp=event.created_at, 
    print_messages=True,
    rich_print=True,
    timeout=120,
)

╭───────────────────────── USER [10/01/2025 23:59:53] ─────────────────────────╮
│ Tell me about recent AI news for today only.                                 │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [10/02/2025 00:00:01] ─────────────────────────╮
│ 🧠 Reasoning                                                                 │
│                                                                              │
│ Searching for recent AI news                                                 │
│                                                                              │
│ The user is asking for recent news about AI today, specifically for October  │
│ 1, 2025. I need to use the web search tool to fetch this information. My     │
│ search query will be "AI news October 1 2025." I’ll ensure to use the        │
│ correct model settings and specify the context for the search. Once I gather │
│ the results, I will provide a concise summary along with sources and links   │
│ for the user.                                                                │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [10/02/2025 00:00:03] ─────────────────────────╮
│ 🔧 Tool Request: openai_web_search                                           │
│                                                                              │
│ Arguments:                                                                   │
│                                                                              │
│                                                                              │
│  {                                                                           │
│    "input": "AI news October 1 2025",                                        │
│    "model": "gpt-5-mini",                                                    │
│    "reasoning_effort": "low",                                                │
│    "type": "web_search_preview_2025_03_11",                                  │
│    "search_context_size": "medium"                                           │
│  }                                                                           │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [10/02/2025 00:00:29] ─────────────────────────╮
│ ✅ Tool Response: openai_web_search                                          │
│                                                                              │
│ Response:                                                                    │
│                                                                              │
│                                                                              │
│  {                                                                           │
│    "type": "text",                                                           │
│    "text": "Here are the main AI-related headlines from October 1, 2025      │
│  (brief summary + sources). Want more depth on any of these? Tell me which   │
│  one.\n\n- Microsoft launches Microsoft 365 Premium (with Copilot integrate  │
│  for individuals) \u2014 Microsoft announced a $19.99/month individual plan  │
│  that bundles Copilot across Office apps, adds higher Copilot usage limits   │
│  and exclusive AI features (Researcher, Analyst, Actions), 1 TB cloud        │
│  storage, and Defender advanced security; Microsoft will retire Copilot Pro  │
│  and encourage migration to the new Premium plan.                            │
│  ([reuters.com](https://www.reuters.com/technology/microsoft-launches-ai-po  │
│  red-365-premium-bundle-1999-per-month-2025-10-01/?utm_source=openai))\n\n-  │
│  Google rolls out \u201cGemini for Home\u201d (major upgrade to Google       │
│  Assistant / Nest) \u2014 Gemini replaces Google Assistant on Google and Ne  │
│  smart speakers (back to original Google Home), adds more                    │
│  conversational/context-aware voice capabilities, new voices, support for    │
│  multiple household profiles, and a premium \u201cGemini Live\u201d          │
│  experience tied to a Google Home Premium subscription. Early Access began   │
│  October 1, with wider rollout planned later.                                │
│  ([theverge.com](https://www.theverge.com/tech/788102/gemini-for-home-new-g  │
│  gle-assistant-launch-date-price-features?utm_source=openai))\n\n- OpenAI    │
│  announces supply agreements/partnerships to support its \u201cStargate\u20  │
│  infrastructure \u2014 reporting says OpenAI has deals with Samsung and SK   │
│  Hynix for memory chips and is planning expanded data-center capacity in     │
│  South Korea to support its large-scale model infrastructure.                │
│  ([theverge.com](https://www.theverge.com/news/789687/openai-samsung-starga  │
│  -chips?utm_source=openai))\n\n- U.S. federal government shutdown begins (O  │
│  1) \u2014 the federal shutdown that started October 1, 2025, has furloughe  │
│  many federal employees and affected agency operations; several public-heal  │
│  and research agencies face partial or full suspensions, which could disrup  │
│  government-funded AI research, grant processing, and collaboration program  │
│  (context: shutdown took effect Oct 1).                                      │
│  ([en.wikipedia.org](https://en.wikipedia.org/wiki/2025_United_States_feder  │
│  _government_shutdown?utm_source=openai))\n\n- AI-generated / AI-directed    │
│  film teaser released \u2014 an AI-directed feature (reported as the first   │
│  full-length AI-generated film) released its first teaser on October 1,      │
│  drawing attention to AI-created media and related debates over credits,     │
│  copyright, and \u201cAI actors.\u201d                                       │
│  ([en.wikipedia.org](https://en.wikipedia.org/wiki/The_Sweet_Idleness?utm_s  │
│  rce=openai))\n\nNotes and next steps\n- I focused on the biggest, widely    │
│  reported items from Oct 1. I can: (a) expand any story into more detail     │
│  (policy, technical, market impact), (b) fetch additional articles or        │
│  reactions (e.g., industry statements or analyst takes), or (c) do a tailor  │
│  roundup for developers, busin

╭──────────────────────── AGENT [10/02/2025 00:00:32] ─────────────────────────╮
│ 🧠 Reasoning                                                                 │
│                                                                              │
│ Summarizing recent AI news                                                   │
│                                                                              │
│ The tool provided aggregated headlines for today, October 1, 2025. I’ll      │
│ present these as concise bullet points with short summaries and source       │
│ links, ensuring I avoid heavy formatting, as suggested. I’ll use terms like  │
│ "reported" to maintain accuracy and avoid definitive claims when uncertain.  │
│ I'll also highlight significant aspects, like the impact of shutdowns on AI  │
│ research and mention "OpenAI supply agreements" potentially from The Verge.  │
│ Keeping to today’s news only, I’ll create a clean presentation with the      │
│ bullet points.                                                               │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [10/02/2025 00:00:34] ─────────────────────────╮
│ 🧠 Reasoning                                                                 │
│                                                                              │
│ Crafting final answer                                                        │
│                                                                              │
│ I should ask the user if they’d like more depth on the information provided. │
│ It's important to keep my final answer clear and concise while using bullet  │
│ points with links. I also want to ensure I include a friendly offer to       │
│ provide more detail if they need it. This way, I’m encouraging engagement    │
│ without overwhelming them. Keeping everything brief will make it easier for  │
│ the user to digest the information!                                          │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [10/02/2025 00:00:36] ─────────────────────────╮
│ Here are the main AI stories reported today (Oct 1, 2025), with brief        │
│ summaries and sources — tell me which one you want expanded.                 │
│                                                                              │
│  • Microsoft launches Microsoft 365 Premium with Copilot for individuals —   │
│    New $19.99/month individual plan bundles Copilot across Office apps,      │
│    higher Copilot usage limits, additional AI features and 1 TB storage;     │
│    Microsoft is encouraging migration from Copilot Pro to this Premium tier. │
│    Source: Reuters (Oct 1, 2025).                                            │
│    Link:                                                                     │
│    https://www.reuters.com/technology/microsoft-launches-ai-powered-365-prem │
│    ium-bundle-1999-per-month-2025-10-01/                                     │
│  • Google rolls out “Gemini for Home” (major Google Assistant / Nest         │
│    upgrade) — Gemini replaces the Google Assistant on Google/Nest smart      │
│    devices with more conversational, context-aware voice capabilities, new   │
│    voices and household-profile support; Early Access begins Oct 1 with      │
│    broader rollout to follow. Source: The Verge (Oct 1, 2025).               │
│    Link:                                                                     │
│    https://www.theverge.com/tech/788102/gemini-for-home-new-google-assistant │
│    -launch-date-price-features                                               │
│  • OpenAI reported to sign supply/partnership deals to scale its             │
│    infrastructure (“Stargate” support) — Reports say OpenAI has agreements   │
│    (e.g., with Samsung, SK Hynix) for memory/chip supply and is expanding    │
│    datacenter capacity (including in South Korea) to support larger model    │
│    deployments. Source: The Verge (Oct 1, 2025).                             │
│    Link: https://www.theverge.com/news/789687/openai-samsung-stargate-chips  │
│  • U.S. federal government shutdown begins — The shutdown that took effect   │
│    today has furloughed many federal staff and disrupted agency operations;  │
│    observers warn it could delay or interrupt government-funded AI research, │
│    grants, and interagency collaborations. (Context and reporting Oct 1,     │
│    2025).                                                                    │
│    Context link:                                                             │
│    https://en.wikipedia.org/wiki/2025_United_States_federal_government_shutd │
│    own                                                                       │
│  • First teaser from an AI-generated / AI-directed feature released — A      │
│    high-profile AI-created film project released its first teaser today,     │
│    stoking debate over credits, copyright and “AI actors.” (Reported Oct 1,  │
│    2025).                                                                    │
│    Reference: https://en.wikipedia.org/wiki/The_Sweet_Idleness               │
│                                                                              │
│ Would you like more detail on any item (technical implications, business     │
│ impact, analyst reaction, full-article excerpts, or additional sources)?     │
╰──────────────────────────────────────────────────────────────────────────────╯